### Problem 4
In this question you will import data from the CSV file created in the Setup section above.

### Problem 4
This data represents the condition of an electrical mini-substation based on readings from voltage, current and temperature sensor readings. A condition of ‘0’ represents a properly functioning device, while a condition of ‘1’ represents failure.

### Problem 4
#### a) Which is worse for this use case, a false positive or a false negative? What value of β would be suitable for an Fβ score?

Let's take positive (a condition of "1") as a failure in device and negative (a condition of "0") as a properly functioning device.
For this use case, a false negative is worse.

A false positive will trigger the alert to the engineers. They will go and check but only to find out the electrical mini-substation is functioning properly. Although resources are used to do this check, it is still fine.

A false negative will cover up a failure in device and let the engineers think that the device is functioning properly. They cannot fix the failure in time. As a result, the electrical mini-substation may fail entirely and the loss can be huge.

Recall measures how many of positive points were correctly predicted by the model. Recall is low when the model creates many false negatives. Recall is a useful metric in models in which we don’t want many false negatives. Since higher values of β give recall more importance, and lower values of β give precision more importance, we would take a higher value of β. A value of 10 for β would be suitable for an Fβ score.

### Problem 4
#### b) Load the CSV file into an SFrame named data. Print the SFrame. Split the data into training/validation/testing sets using 80%/10%/10% respectively.

In [50]:
import turicreate as tc
import numpy as np
import random

In [51]:
data = tc.SFrame("0380915_data.csv")

Finished parsing file /home/jryk/assignment_3/0380915_data.csv

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,float,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Parsing completed. Parsed 100 lines in 0.021123 secs.

Finished parsing file /home/jryk/assignment_3/0380915_data.csv

Parsing completed. Parsed 1000 lines in 0.005906 secs.

In [52]:
data

Condition,Voltage,Current,Temperature
1,25.666912748577367,443.1776787138455,39.04298008669625
1,24.93619269601364,443.7752058654272,36.615609013862766
1,26.431378686661073,440.70519661137064,59.53153475566279
1,25.75551597941211,444.45465600218176,42.75831261815376
1,25.97123695471139,442.8046424128778,37.65621788818368
1,25.88350340890212,444.5814752212545,39.4083109258523
0,26.94535437487196,442.3878725340347,45.120200601609525
1,26.490910090226947,444.4899725301915,40.08125414679967
1,25.279176510841378,442.3930666295656,38.47134221045628
1,26.083107828134683,441.3197317953561,52.22772264719654


In [53]:
train_data, test_validate_data = data.random_split(.8, seed=0, exact=True)

In [54]:
test_data, validate_data = test_validate_data.random_split(.5, seed=0, exact=True)

In [55]:
# check the length of training set
print("Length of training set:", len(train_data))
print("Length of validation set:", len(validate_data))
print("Length of testing set:", len(test_data))

Length of training set: 800
Length of validation set: 100
Length of testing set: 100


### Problem 4
#### c) Is feature rescaling turned on by default for the function turicreate.logistic_classifier.create? What scale are the coefficients given in?

According to the Turi Create User Guide, feature rescaling is turned on by default. [feature_rescaling=True]

The coefficients are given in original scale of the problem. 

https://apple.github.io/turicreate/docs/userguide/supervised-learning/linear-regression.html#feature-rescaling

### Problem 4
#### d) Create perceptrons using Turicreate to classify data with ‘Condition’ as the target. Be sure to use your validation set in the training. Experiment with different values of hyperparameters to develop two different models.

In [56]:
perceptron_01 = tc.logistic_classifier.create(
    train_data, target="Condition", 
    l2_penalty=0.01, l1_penalty=0, 
    validation_set=validate_data, seed=0)

Logistic regression:

--------------------------------------------------------

Number of examples          : 800

Number of classes           : 2

Number of feature columns   : 3

...

| 2         | 3        | 0.002584     | 0.906250          | 0.860000            |

| 3         | 4        | 0.003618     | 0.906250          | 0.860000            |

| 4         | 5        | 0.004584     | 0.906250          | 0.860000            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [57]:
perceptron_01.coefficients

name,index,class,value,stderr
(intercept),None,1,-1.2478864522283428,4.671650565067614
Voltage,None,1,0.10393502860364201,0.1606706706928269
Current,None,1,-0.0008625299802514587,0.0010394528439117846
Temperature,None,1,0.022418916249192358,0.014313969542919608


In [60]:
results = perceptron_01.evaluate(test_data)

In [61]:
print("Accuracy         : %s" % results['accuracy'])
print("Confusion Matrix : \n%s" % results['confusion_matrix'])

Accuracy         : 0.89
Confusion Matrix : 
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+
|      0       |        1        |   5   |
|      1       |        1        |   44  |
|      0       |        0        |   45  |
|      1       |        0        |   6   |
+--------------+-----------------+-------+
[4 rows x 3 columns]



In [58]:
predictions = perceptron_01.predict(test_data)

In [62]:
# Compute boolean filters
false_positive_filter = (predictions == 1) & (test_data[perceptron_01.target] == 0)
false_negative_filter = (predictions == 0) & (test_data[perceptron_01.target] == 1)

false_negatives = test_data[false_negative_filter]
false_positives = test_data[false_positive_filter]

In [63]:
perceptron_02 = tc.logistic_classifier.create(train, target="Condition", l2_penalty=0.1, l1_penalty=0.1, validation_set=validate, seed=0)

Logistic regression:

--------------------------------------------------------

Number of examples          : 800

Number of classes           : 2

Number of feature columns   : 3

...

| 5         | 6        | 0.003425  | 0.015427     | 0.906250          | 0.860000            |

| 10        | 11       | 0.003425  | 0.021841     | 0.906250          | 0.860000            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [64]:
perceptron_02

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 4
Number of examples             : 800
Number of classes              : 2
Number of feature columns      : 3
Number of unpacked features    : 3

Hyperparameters
---------------
L1 penalty                     : 0.1
L2 penalty                     : 0.1

Training Summary
----------------
Solver                         : fista
Solver iterations              : 10
Solver status                  : Completed (Iteration limit reached).
Training time (sec)            : 0.0225

Settings
--------
Log-likelihood                 : 248.7527

Highest Positive Coefficients
-----------------------------
(intercept)                    : 0.1793
Voltage                        : 0.0957

Lowest Negative Coefficients
----------------------------
Temperature                    : -0.007
Current                        : -0.0004

In [65]:
perceptron_02.evaluate(test)

{'accuracy': 0.89,
 'auc': 0.9222000000000001,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |   5   |
 |      1       |        1        |   44  |
 |      0       |        0        |   45  |
 |      1       |        0        |   6   |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.888888888888889,
 'log_loss': 0.34607038140495633,
 'precision': 0.8979591836734694,
 'recall': 0.88,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+-----+-----+----+----+
 | threshold | fpr | tpr | p  | n  |
 +-----------+-----+-----+----+----+
 |    0.0    | 1.0 | 1.0 | 50 | 50 |
 |   0.001   | 1.0 | 1.0 | 50 | 50 |
 |   0.002   | 1.0 | 1.0 | 50 | 50 |
 |   0.003   | 1.0 | 1.0 | 50 | 

### Problem 4
#### e) For each model:
##### i) find predictions using the testing set;
##### ii) display the training/validation/testing accuracy;
##### iii) display the confusion matrix on the testing set;
##### iv) calculate recall, precision, sensitivity and specificity on the testing set;
##### v) calculate the Fβ score on the testing set using the value of β you chose above.

### Problem 4
#### f) Select which of your two models is the best (or declare a tie) and justify your choice.